## Amsterdam Science Puzzle

The following puzzle was posted by the [Amsterdam Science](http://amsterdamscience.org/) magazine.
Alhtough it is meant to be solved in a different way, I choose to write few codes to help me with that. 

Here is the problem:
<img src="puzzle.png">

Over the next steps, you will follow what I did. If you have suggestions to improve the solution, let me know!


## 1 - English Word's List
    I found this [github repository](https://github.com/dwyl/english-words) several files containing a list of English words. For this example, I am just considering words that do not contain numbers.  


In [41]:
# import libraries
import requests
import os.path

# Path to list of words
url = 'https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt'

def download_data():
    if os.path.isfile('./words_alpha.txt'):
        print('File exists. Skipping download')
    else:
        r = requests.get(url, stream=True)
        with open('words_alpha.txt', 'wb') as f:
            for chunk in r.iter_content():
                f.write(chunk)
        print("Download finished.")

## Load all words
def load_words():
    download_data()

    with open('words_alpha.txt') as word_file:
        valid_words = set(word_file.read().split())

    return valid_words

english_words = load_words()
print('The list of words contain {} words'.format(sum(1 for line in english_words)))
    

File exists. Skipping download
The list of words contain 370099 words


That's quite a large list of words. However, for this problem in particular we just need the words that contain 5 or 8 characters.

## Selecting words with 5 or 8 characters


In [42]:
def select_words_by_size(list_of_words, size):
    new_list = [word for word in english_words if len (word) == size]
    print('The new list contain {} words with {} letters'.format(sum(1 for line in new_list), size))
    return new_list

words_5_letters = select_words_by_size(english_words, 5)
words_8_letters = select_words_by_size(english_words, 8)

The new list contain 15918 words with 5 letters
The new list contain 51626 words with 8 letters


## Finding the new words

The problem gives a list of words containing 4 letters. To each of these words, we should be able to add 1 more leeter and form a new word by rearranging the letters.



In [44]:
# List of words provided by the problem.

words = ['thin',
        'rind',
        'site',
        'toga',
        'swat',
        'mail',
        'city',
        'deny']


Now, we have to find all the 5-letter words that contain the 4-letter words. 

In [54]:
def matching_5_words(words, words_5_letters):
    new_words = {}

    # Create a dictionary to make the next loops smaller
    for word in words:
        letters = list (word)
        # x = []
        x = [word for word in words_5_letters if all (letter in word for letter in letters)]
        new_words[word] = x
    return new_words

new_words = matching_5_words(words, words_5_letters)

for key, value in new_words.items():
    print("We found {} possibilites for the word {}".format(len(value), key))


We found 5 possibilites for the word city
We found 20 possibilites for the word deny
We found 30 possibilites for the word swat
We found 20 possibilites for the word toga
We found 17 possibilites for the word thin
We found 37 possibilites for the word mail
We found 19 possibilites for the word rind
We found 87 possibilites for the word site


That's quite a large ammount of possibilities! Just for the word **city**, we can for 5 new words by adding a letter. For words like **site**, we can form 87 new 5-letter wokrds.

Let's check the solution for **city** words:

In [56]:
print(new_words['city'])

['itchy', 'typic', 'lytic', 'dicty', 'ticky']


However, this is part of the solution. We want to know also which letter was added to the 4-letter word to form the 5-letter word.

In [70]:
import string
from collections import Counter
from collections import defaultdict

added_letters = {}

def match_words(word1, word2):
    bag = Counter (word1)
    bag.subtract (Counter (word2))

    if all (v == 0 for v in bag.values ()):
        return True
    else:
        return False
    
for key, value in new_words.items ():
    print (key.upper ())
    valid_letters = []

    for char in string.ascii_lowercase:
        expanded_word = key+char

        # After adding a letter to the end, get the list of matching words
        matching_words = [word for word in value if all (letter in word for letter in list(expanded_word))]

        matches_per_letter = [word for word in matching_words if match_words (expanded_word, word)]
        if len(matches_per_letter) > 0 :
            valid_letters.append(char)
            print("{} : {}".format(char, matches_per_letter))

    added_letters[key] = valid_letters


CITY
d : ['dicty']
h : ['itchy']
k : ['ticky']
l : ['lytic']
p : ['typic']
DENY
a : ['denay']
b : ['bendy']
d : ['neddy']
e : ['deeny', 'needy']
f : ['fendy']
h : ['hynde', 'hendy']
k : ['kendy']
l : ['dynel']
m : ['mendy']
o : ['doney', 'doyen']
s : ['snyed', 'dynes']
t : ['deynt', 'tyned', 'denty']
u : ['undye']
w : ['wendy']
SWAT
a : ['wasat']
d : ['datsw', 'dawts']
e : ['tawse', 'twaes', 'sweat', 'waste', 'etwas', 'awest']
f : ['wafts']
h : ['swath', 'whats', 'thaws']
i : ['waist', 'swati', 'waits']
n : ['wants', 'stawn', 'wasnt']
r : ['wrast', 'swart', 'warst', 'straw', 'starw', 'warts']
s : ['swats', 'wasts']
t : ['twats', 'watts']
y : ['wasty']
TOGA
b : ['tabog']
c : ['cagot']
e : ['togae']
f : ['fagot']
h : ['gotha']
l : ['gloat']
m : ['magot']
n : ['tango', 'tonga']
r : ['groat', 'gator', 'argot', 'gotra']
s : ['goats', 'togas', 'stoga']
t : ['gotta']
u : ['guato']
v : ['gavot']
y : ['goaty']
THIN
a : ['ahint', 'hiant', 'tahin']
c : ['nitch', 'nicht', 'chint']
e : ['thine', 't

In [71]:
# Counting and checking the combinations
for key, value in added_letters.items ():
    print("{} possibilities for  {}  : {}".format(len(value), key, value))

23 possibilities for  site  : ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
5 possibilities for  city  : ['d', 'h', 'k', 'l', 'p']
11 possibilities for  swat  : ['a', 'd', 'e', 'f', 'h', 'i', 'n', 'r', 's', 't', 'y']
14 possibilities for  toga  : ['b', 'c', 'e', 'f', 'h', 'l', 'm', 'n', 'r', 's', 't', 'u', 'v', 'y']
14 possibilities for  deny  : ['a', 'b', 'd', 'e', 'f', 'h', 'k', 'l', 'm', 'o', 's', 't', 'u', 'w']
17 possibilities for  mail  : ['a', 'b', 'c', 'd', 'e', 'h', 'i', 'k', 'l', 'm', 'n', 'p', 'r', 's', 't', 'u', 'x']
10 possibilities for  rind  : ['a', 'e', 'g', 'i', 'k', 'n', 'o', 's', 'u', 'y']
9 possibilities for  thin  : ['a', 'c', 'e', 'g', 'k', 'n', 'o', 's', 'u']


### Partial solution

Ok! We found all possible inputs for the boxes at the end. In principle, you could think in 103 words that might fit the first condition of the problem. That's quite a lot already for a "simple" puzzle.

If you try then to find the new 8-letter word by simply combining the added letter, that will be a massive effort. There are 379348200 combinations possible! Yes. Almost 380 million combinations. 

Sure. Not all these combinations produce a valid word. And our brain would filter many possibilities reducing a lot the timing an effort. 

### What are the valid words then?

We have to plug all these results together. The letters forming the 5-letter word from the 4-letter word and the anagrams.

The solution is quite simple for the computer. We will look for all the words in the anagrams list that start with a letter added to **thin**. 

This means that all 8-letter words that start with 'a', 'c', 'e', 'g', 'k', 'n', 'o', 's', 'u' will be kept.
Then, from the previous step, we get kept all letters with 'a', 'e', 'g', 'i', 'k', 'n', 'o', 's', 'u', 'y' in the second position. This is the possible additions for  **rind**. 


We do that for all words, and we should have our answer.

In [142]:
def filter_words(list_words, position, list_letters):
    answers = []
    for word in list_words:
        for l in list_letters:
            if word[position] == l:
                answers.append (word)

    return answers


def letter_in_position(list_words, words):
    print('We start with {} 8-letter words'.format(len(list_words)))
    
    for index, word in enumerate(words):
        list_words = filter_words(list_words, index, added_letters[word])
        print('{} 8-letter words after filtering with added letters to {}'.format(len(list_words), word))
       
    return list_words

answer_8_letter = letter_in_position(words_8_letters, words )


We start with 51626 8-letter words
23475 8-letter words after filtering with added letters to thin
13418 8-letter words after filtering with added letters to rind
12944 8-letter words after filtering with added letters to site
7438 8-letter words after filtering with added letters to toga
4897 8-letter words after filtering with added letters to swat
4155 8-letter words after filtering with added letters to mail
526 8-letter words after filtering with added letters to city
384 8-letter words after filtering with added letters to deny


There are still 384 words!!! 


## 8-letter Anagrams


Finally, the letters added should form an Anagram. By definition, Anagram is a word, phrase, or name formed by rearranging the letters of another, such as spar, formed from rasp.

Check on [Wikipedia](https://en.wikipedia.org/wiki/Anagram) for more information.

Let's filter our pool of 8-letter words by selection all that are anagrams! 

In [143]:
def get_anagrams(source):
    ## Populate dictionary with word combinations
    d = {}
    for word in source:
        key = "".join(sorted(word))
        if key in d.keys():  
            d[key].append(word)  
        else:  
            d[key] = [word]  
    
    ## Get the combinations that just yeld 1 word, ie. not anagrams
    keys = []
    count = 0
    for key, value in d.items():
        if(len(value)==1):
            keys.append(key)
        else:
            count += len(value)
    
    #remove from the list and return the anagrams
    for k in keys:
        del d[k]
        

    return d, count

# Get the dictionary of anagrams
anagrams , count = get_anagrams(answer_8_letter)
print('We found {} anagrams'.format(count))

We found 6 anagrams


## The Solution

Not bad. We finally found just 6 anagrams that are solutions. Let's check then:

In [146]:
anagrams

{'aabeells': ['sealable', 'saleable'],
 'aabeelnr': ['earnable', 'nearable'],
 'aeillnss': ['ainsells', 'sensilla']}

So, this means that, you could form any of these words and have the problem right.

Below, I'll provide one full example, but remind that we found thousands of possible combinations. 

| 4-letter Word | Letter Added | 5-letter word| Anagram |
|------|--------------|----------|---------|
| Thin | e            | Ahint    | n       |
| Rind | a            | Drain    | e       |
| Site | r            | Tires    | a       |
| Toga | n            | Tango    | r       |
| Swat | a            | Wasat    | a       |
| Mail | b            | Limba    | b       |
| City | l            | Lytic    | l       |
| Deny | e            | Needy    | e       |

## Final comments

I didn't imagine to find so many possible combinations. These answers are sure limited by the list of words I used and many other solutions are possible. Which ones did you find?
